# nPE: A Configurable Processing Engine
#### Documentation | Version: 0.4.1 | Updated 2018.7.25
---

## Introduction

The nPE is a highly configurable processing engine that can be topologically configured to support various DNN accelerator architectures. Its functionality is able to support inner products with varying amounts of spatial and temporal parallelism, nonlinear activation functions, scalar addition, scalar multiplication, and max functions. It supports many existing architectures and dataflows, such as Eyeriss, Flexflow, and ShiDianNao. Its generality based on the notion of soft, firm, and hard bypasses.
- Soft bypasses make it possible to bypass or not bypass via control signals and a multiplexer.
- Firm bypasses add a bypass line as an output.
- Hard bypasses hardwire the input and output together.

The details of bypass behavior change depending on the specific module.

It's microarchitecture consists of two multiport Register Files, an n-dimensional inner product unit, an RF-with-ALU combo, and a nonlinear processing unit. All three registers are accessible from an external NoC, and all arithmetic modules support an "Identity" operation.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = """
C:\Users\RyanL\OneDrive\Research\SEAL\processing-engine/source/load-ivy.sc
"""

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import scala.math.pow

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}


import scala.math.pow

## Parallel Register File

The Parallel Register File (pRF) is a register file that is configurable based on number of ports in/out, register size, bypass type, and bitwidth. Bypass options are None, Soft, or Hard. The bypass behavior is such that it bypasses in[k] to out[k], where k is the port index. Without loss of generality, the bypass option will be applied to all ports. Multiple bypass options are enabled by parallel pRFs.

### Single Register File

The Single Register File acts as expected: it reads data in when write enable is high and reads data when read enable is high. It can read and write in parallel. It has two read ports: one for internal PE reading and one for external PE communication. The register size adjusts based on the bitwidth of the addresses.

In [3]:
class RFControl(addrWidth: Int, bpSupp: Boolean) extends Bundle {
    
    override def cloneType = (new RFControl(addrWidth, bpSupp)).asInstanceOf[this.type]
    
    val wEnable  = Bool()
    val rEnable  = Bool()
    val wAddr    = UInt(addrWidth.W)
    val rAddrInt = UInt(addrWidth.W)
    val rAddrExt = UInt(addrWidth.W)
    val bpSel    = if (bpSupp) Some(Bool()) else None
}

class RFOutputs(dataWidth: Int) extends Bundle {
    
    override def cloneType = (new RFOutputs(dataWidth)).asInstanceOf[this.type]
    
    val int = SInt(dataWidth.W)
    val ext = SInt(dataWidth.W)
}

class RF(dataWidth: Int, addrWidth: Int, bpSupp: Boolean) extends Module {
  
    val io = IO(new Bundle {
        val control = Input(new RFControl(addrWidth, bpSupp))
        val dataIn  = Input(SInt(dataWidth.W))
        val dataOut = Output(new RFOutputs(dataWidth))
    })
    
    val registers = RegInit(Vec(Seq.fill(pow(2, addrWidth).toInt) { 0.S(addrWidth.W) }))
    
    when (io.control.wEnable) {
        registers(io.control.wAddr) := io.dataIn
    }
    
    when (io.control.rEnable) {
        when(io.control.bpSel.getOrElse(false.B)) {
            io.dataOut.int := io.dataIn
            io.dataOut.ext := io.dataIn
        } .otherwise {
            io.dataOut.int := registers(io.control.rAddrInt)
            io.dataOut.ext := registers(io.control.rAddrExt)
        }
    } .otherwise {
        io.dataOut.int := 0.S
        io.dataOut.ext := 0.S
    }
}

defined class RFInputs
defined class RFOutputs
defined class RF

### Putting them Together

The single RFs come together in parallel to form the pRF. The number of ports is the number of single RFs. Thus, there are all of the control signals associated with a single RF, but with a bus width equal to the port count. If the bypass type is Soft, then each RF has an independent bypass control signal.

In [4]:
class PRFControl(ports: Int, addrWidth: Int, bpSoft: Boolean) extends Bundle {
    
    override def cloneType = (new PRFControl(ports, addrWidth, bpSoft)).asInstanceOf[this.type]
    
    val rf = Vec(ports, new RFControl(addrWidth, bpSoft))
}

class PRF(ports: Int, dataWidth: Int, addrWidth: Int, bp: String) extends Module {
    
    require(List("None", "Soft", "Hard") contains bp)
    
    val bpNone = (bp == "None")
    val bpSoft = (bp == "Soft")
    val bpHard = (bp == "Hard")
    
    val io = IO(new Bundle {
        val control = Input(new PRFControl(ports, addrWidth, bpSoft))
        val dataIn = Input(Vec(ports, SInt(dataWidth.W)))
        val dataOut = Output(Vec(ports, new RFOutputs(dataWidth)))
    })
    
    if (bpNone || bpSoft) {
        
        val rf = Seq.fill(ports){ Module(new RF(dataWidth, addrWidth, bpSoft)) }
        
        rf.zipWithIndex.map {
            case (x: RF, i: Int) => { 
                x.io.control <> io.control.rf(i)
                x.io.dataIn := io.dataIn(i)
                
                io.dataOut(i) <> x.io.dataOut
            } 
        }
        
    } else if (bpHard) {
        for (i <- 0 until ports) {
            io.dataOut(i).int := io.dataIn(i)
            io.dataOut(i).ext := io.dataIn(i)
        }
    }
}

defined class PRFInputs
defined class PRFOutputs
defined class PRF

## Inner Product Unit

The Inner Product Unit (IPU) performs an inner product between two vectors of a configurable length. For SIMD support, this should be configured to be wide. For traditional PEs, such as Eyeriss, note that setting the width to 1 is equivalent to a scalar multiplication.

Bypass options are None or Firm. For Firm, there only one pair of weights and activations may be bypassed at a time, such that the IPU may interface with only one ALU. Thus if the bypass type is Firm, then there is a bypass selection signal that selects which input to feed to the output. This may be generalized to interface with multiple TUs, but not at this time.

There are plans to implement both Soft and Hard bypassing. There are also plans to make using FMA modules an option.

The IPU contains two embedded modules: a parallel multiplier and a reduction tree.

### Parallel Multiplier

The parallel multiplier simply takes two vector inputs of width n and outputs the element-wise multiplication. There are plans to make the multiplier type configurable, *e.g.* serial, combinational, pipelined *etc*.

In [5]:
def checkParamsPMult(numPairs: Int, bitWidth: Int) {
    require(numPairs >= 1, "Must have at least one pair of multiplicands.")
    require(bitWidth >= 1, "Bitwidth must be at least one.")
}

class PMultInputs(numPairs: Int, bitWidth: Int) extends Bundle {
    val left  = Input(Vec(numPairs, SInt(bitWidth.W)))
    val right = Input(Vec(numPairs, SInt(bitWidth.W)))
}

class PMult(numPairs: Int, bitWidth: Int) extends Module {
    
    val io = IO(new Bundle {
        val in   = new PMultInputs(numPairs, bitWidth)
        val prod = Output(Vec(numPairs, SInt(bitWidth.W)))
    })
    
    io.prod := (io.in.left zip io.in.right).map { case(a, b) => a * b }
}

defined function checkParamsPMult
defined class PMultInputs
defined class PMult

### Additive Reduction Tree

The additive reduction tree outputs the L1 norm of the n-dim input vector using log(n) layers of 2-input adders. In other words, it sums the input.

There are plans to pipeline this (as well as everything else...).

In [6]:
// Recursively creates a balanced syntax tree
def nonassocPairwiseReduce[A](xs: List[A], op: (A, A) => A): A = {
  xs match {
    case Nil => throw new IllegalArgumentException
    case List(singleElem) => singleElem
    case sthElse => {
      val grouped = sthElse.grouped(2).toList
      val pairwiseOpd = for (g <- grouped) yield {
        g match {
          case List(a, b) => op(a, b)
          case List(x) => x
        }
      }
      nonassocPairwiseReduce(pairwiseOpd, op)
    }
  }
}

def checkParamsAdditiveRT(numAddends: Int, bitWidth: Int): Unit = {
    require(numAddends >= 1, "Number of addends must be at least one.")
    require(bitWidth >= 1, "Bitwidth must be at least one.")
}


class AdditiveRT(numAddends: Int, bitWidth: Int) extends Module {

    checkParamsAdditiveRT(numAddends, bitWidth)
    
    val io = IO(new Bundle {
        val in  = Input(Vec(numAddends, SInt(bitWidth.W)))
        val sum = Output(SInt(bitWidth.W))
    })
    
    io.sum := nonassocPairwiseReduce(io.in toList, (x: SInt, y: SInt) => x + y)
}

defined function nonassocPairwiseReduce
defined function checkParamsAdditiveRT
defined class AdditiveRT

### Putting them Together

Together, the parallel multiplier and additive reduction tree perform an n-dim inner product. They are simply connected output to input. During use of the PE, the bypass type of the IPU is dictated by the functionality of the ALU.

In [7]:
def checkParamsIPU(width: Int, bitWidth: Int, bp: String) {
    val bpError = "Bypass must be \"None\" or \"Firm\""
    val widthError = "Width must be at least one"
    val bitWidthError = "Data bitwidth must be non-negative"
    
    val bpTypes = List("None", "Firm")
    
    require(width >= 1, widthError)
    require(bpTypes.contains(bp), bpError)
    require(bitWidth >= 0, bitWidthError)
}

class IPUInputs(width: Int, bitWidth: Int, bpFirm: Boolean) extends Bundle {
    val data = new PMultInputs(width, bitWidth)
    val sel  = if (bpFirm) Some(Input(Vec(width, Bool()))) else None
}

class IPUOutputs(bitWidth: Int, bpFirm: Boolean) extends Bundle {
    val innerProd = Output(SInt(bitWidth.W))
    
    val bpLeft  = if (bpFirm) Some(Output(SInt(bitWidth.W)))  else None
    val bpRight = if (bpFirm) Some(Output(SInt(bitWidth.W)))  else None
}


class IPU(width: Int, bitWidth: Int, bypassType: String) extends Module {
    
    checkParamsIPU(width, bitWidth, bypassType)
    
    val bpFirm = (bypassType == "Firm")
    
    val io = IO(new Bundle {
        val in  = new IPUInputs(width, bitWidth, bpFirm)
        val out = new IPUOutputs(bitWidth, bpFirm)
    })
    
    val pMult = Module(new PMult(width, bitWidth))
    pMult.io.in <> io.in.data
    
    val additiveRT = Module(new AdditiveRT(width, bitWidth))
    additiveRT.io.in := pMult.io.prod
    
    io.out.innerProd := additiveRT.io.sum
    
    if (bpFirm) {
        io.out.bpLeft.get  := PriorityMux(io.in.sel.get, io.in.data.left)
        io.out.bpRight.get := PriorityMux(io.in.sel.get, io.in.data.right)
    }
}

defined function checkParamsIPU
defined class IPUInputs
defined class IPUOutputs
defined class IPU

## ALU

The ALU is configurable based on the functions it should support and the data bitwidth. The minimum ALU simply connects the input and the output directly. Accumulate means add the local register file output to the inner product from the IPU. Add and Max do perform their respective operation on the two bypasses from the IPU.

Selecting the operation to perfom is done with a one-hot select signal. The order is always Identity, Add, Max, then Accumulate, but the exact indices will change depending on which are supported.

There are plans to add bypass behavior. Right now, it is supported with an identity operation.

In [8]:
def checkparamsALU(datawidth: Int, funcs: List[String]) {
    val identityError = "ALU functions must explicitly include Identity."
    val functionError = "Unsupported Error"
    val supportedFuncs = List("Identity", "Add", "Max", "Accumulate")
    
    require(funcs.contains("Identity"), identityError)
    for(x <- funcs) { require(supportedFuncs.contains(x), functionError) }
}

class ALUInputs(dataWidth: Int, numFuncs: Int, accSupp: Boolean, addBypassIn: Boolean) extends Bundle {
    val innerProd = Input(SInt(dataWidth.W))
    val funcSel = Input(Vec(numFuncs, Bool()))
    
    val weightBp     = if(addBypassIn) Some(Input(SInt(dataWidth.W))) else None
    val actvtnBp     = if(addBypassIn) Some(Input(SInt(dataWidth.W))) else None
    val rfFeedback   = if(accSupp)     Some(Input(SInt(dataWidth.W))) else None
}

class ALU(dataWidth: Int, funcs: List[String]) extends Module {
    
    checkparamsALU(dataWidth, funcs)
    
    val addSupp = funcs.contains("Add")
    val maxSupp = funcs.contains("Max")
    val accSupp = funcs.contains("Accumulate")
    val addBypassIn = addSupp || maxSupp
 
    val io = IO(new Bundle {
        val in = new ALUInputs(dataWidth, funcs.length, accSupp, addBypassIn)
        val out = Output(SInt(dataWidth.W))
    })
    
    val idnOut = Some(Wire(SInt(dataWidth.W)))
    val addOut = if(addSupp) Some(Wire(SInt(dataWidth.W))) else None
    val maxOut = if(maxSupp) Some(Wire(SInt(dataWidth.W))) else None
    val accOut = if(accSupp) Some(Wire(SInt(dataWidth.W))) else None
    
    idnOut.get := io.in.innerProd
    
    if (addSupp) { addOut.get := io.in.weightBp.get + io.in.actvtnBp.get }
    if (accSupp) { accOut.get := io.in.innerProd + io.in.rfFeedback.get }
    if (maxSupp) {
        when (io.in.weightBp.get > io.in.actvtnBp.get) {
            maxOut.get := io.in.weightBp.get
        } .otherwise {
            maxOut.get := io.in.actvtnBp.get
        }
    }
    
    val inters = (idnOut :: addOut :: maxOut :: accOut :: Nil) filter ( _.isDefined ) map ( _.get )
    io.out := PriorityMux(io.in.funcSel, inters)
}

defined function checkparamsALU
defined class ALUInputs
defined class ALU

## Feedback Register File

The feedback register file is just a pRF of size 1 as implemented earlier. Its bypass behavior is key to implementing architectures such as FlexFlow. A Hard bypass will connect the input from the NoC to the NLU/ALU and the input from the ALU to the NoC.

## Nonlinear Unit

The Nonlinear Unit (NLU) performs the nonlinear activations functions present in neural networks. As of right now, it supports the Identity function and ReLu, but there are plans to implement sigmoid and tanh using LUTs and linear interpolation.

In [9]:
def checkparamsNLU(funcs: List[String], datawidth: Int) {
    require(funcs.contains("Identity"), "NLU functions must explicitly include Identity.")
    val supportedFuncs = List("Identity", "ReLu")
    for(x <- funcs)(require(supportedFuncs.contains(x), "Unsupported Function"))
}

class NonlinearUnit(funcs: List[String], datawidth: Int) extends Module {
    
    checkparamsNLU(funcs, datawidth)
    
    val io = IO(new Bundle {
        val input = Input(SInt(datawidth.W))
        val fslct = Input(Vec(funcs.length, Bool()))
        val outpt = Output(SInt(datawidth.W))
    })
    
    val idntOut = Some(Wire(SInt(datawidth.W)))
    val reluOut = if(funcs.contains("ReLu")) Some(Wire(SInt(datawidth.W))) else None
    
    idntOut.get := io.input
    if (funcs.contains("ReLu")) {
        when (io.input > 0.S) {
            reluOut.get := io.input
        } .otherwise {
            reluOut.get := 0.S
        }
    }
    
    val inters = (idntOut :: reluOut :: Nil) filter ( _.isDefined ) map ( _.get )
    io.outpt := PriorityMux(io.fslct, inters)
}

defined function checkparamsNLU
defined class NonlinearUnit

## Control

Control is implemented via a State Machine (Moore) and Decoder. The state machine must have its states and state transitions configured. The Decoder must have its input-output map configured.

### State Machine

The state machine takes a map as a parameter and constructs the appropriate hardware implementation. The State Machine is configured via the "nextState" parameter: a map from the current state and current input to the next state.

In [10]:
class StateMachine(numStates: Int, nextState: (UInt, UInt, Int) => UInt, ctrlWidth: Int) extends Module {
    
    val stateWidth: Int = log2Up(numStates)
    
    val io = IO(new Bundle {
        val control = Input (UInt(ctrlWidth.W ))
        val out     = Output(UInt(stateWidth.W))
    })
    
    val register = RegInit(0.U(stateWidth.W))
    register := nextState(register, io.control, ctrlWidth)
    io.out := register
}

defined class StateMachine

### Decoder

The decoder acts as is typical in processors: it converts the state information into all the control signals associated with the appropriate state. It must be configured via the "decode" parameter, a map from the state and control signal name to the desired value.

In [12]:
class RFControl(ports: Int, addrWidth: Int) extends Bundle {
    
    override def cloneType = (new RFControl(ports, addrWidth)).asInstanceOf[this.type]
    
    val wEnable  = Output(Vec(ports, Bool()))
    val rEnable  = Output(Vec(ports, Bool()))
    val wAddr    = Output(Vec(ports, UInt(addrWidth.W)))
    val rAddrInt = Output(Vec(ports, UInt(addrWidth.W)))
    val rAddrExt = Output(Vec(ports, UInt(addrWidth.W)))
    val bpSelGet = Output(Vec(ports, Bool()))
}

class Decoder(decode: (UInt, String) => Data,
              statewidth: Int,
              ports: Int, datawidth: Int, addrwidth: Int,
              aluFuncs: List[String], 
              nluFuncs: List[String]) extends Module {
    
    val io = IO(new Bundle {
        
        val state = Input(UInt(statewidth.W))
        
        val weightRFControl = new RFControl(ports, addrwidth)
        val actvtnRFControl = new RFControl(ports, addrwidth)
        
        val ipuSelGet       = Output(Vec(ports, Bool()))
        val aluFuncSel      = Output(Vec(aluFuncs.length, Bool()))
        
        val intrnlRFControl = new RFControl(ports, addrwidth)
        val intrnlRFDataSel = Output(Bool())
        
        val nluFuncSel      = Output(Vec(nluFuncs.length, Bool()))
        
    })
    
    // Refactor this to use bulk connections
    io.weightRFControl.wEnable  := decode(io.state, "weightRF wEnable")
    io.weightRFControl.rEnable  := decode(io.state, "weightRF rEnable")
    io.weightRFControl.wAddr    := decode(io.state, "weightRF wAddr")
    io.weightRFControl.rAddrInt := decode(io.state, "weightRF rAddrInt")
    io.weightRFControl.rAddrExt := decode(io.state, "weightRF rAddrExt")
    io.weightRFControl.bpSelGet := decode(io.state, "weightRF bpSelGet")
    
    io.actvtnRFControl.wEnable  := decode(io.state, "actvtnRF wEnable")
    io.actvtnRFControl.rEnable  := decode(io.state, "actvtnRF rEnable")
    io.actvtnRFControl.wAddr    := decode(io.state, "actvtnRF wAddr")
    io.actvtnRFControl.rAddrInt := decode(io.state, "actvtnRF rAddrInt")
    io.actvtnRFControl.rAddrExt := decode(io.state, "actvtnRF rAddrExt")
    io.actvtnRFControl.bpSelGet := decode(io.state, "actvtnRF bpSelGet")
    
    io.ipuSelGet  := decode(io.state, "ipuSelGet")
    io.aluFuncSel := decode(io.state, "aluFuncSel")
    
    io.intrnlRFControl.wEnable  := decode(io.state, "intrnlRF wEnable")
    io.intrnlRFControl.rEnable  := decode(io.state, "intrnlRF rEnable")
    io.intrnlRFControl.wAddr    := decode(io.state, "intrnlRF wAddr")
    io.intrnlRFControl.rAddrInt := decode(io.state, "intrnlRF rAddrInt")
    io.intrnlRFControl.rAddrExt := decode(io.state, "intrnlRF rAddrExt")
    io.intrnlRFControl.bpSelGet := decode(io.state, "intrnlRF bpSelGet")
    io.intrnlRFDataSel          := decode(io.state, "intrnlRF dataSel")
    
    io.nluFuncSel := decode(io.state, "nluFuncSel")
}

defined class Decoder

## Putting it all Together

In [18]:
class nPE(stateMap: Map[(UInt, UInt), UInt], extrnl_ctrl_width: Int, // State Machine
          decode: (UInt, String) => Data, RFports: Int, weightRFBP: String, actvtnRFBP: String, datawidth: Int, addrwidth: Int,
          aluFuncs: List[String], nluFuncs: List[String], intrnlRFBP: String
         ) extends Module {
    
    val io = IO(new Bundle {
        val extrnl_ctrl   = Input (SInt(extrnl_ctrl_width.W))
        val weightRF_in   = Input (Vec(RFports, SInt(datawidth.W)))
        val actvtnRF_in   = Input (Vec(RFports, SInt(datawidth.W)))
        val intrnlRF_in   = Input (SInt(datawidth.W))
        val weightRF_2NoC = Output(Vec(RFports, SInt(datawidth.W)))
        val actvtnRF_2NoC = Output(Vec(RFports, SInt(datawidth.W)))
        val intrnlRF_2NoC = Output(SInt(datawidth.W))
        val output        = Output(SInt(datawidth.W))
    })
    
    val stateMachine = new StateMachine(stateMap, extrnl_ctrl_width)
    stateMachine.io.control := io.extrnl_ctrl
    
    val decoder = new Decoder(decode, log2Up(stateMap.size), 
                              RFports, datawidth, addrwidth, aluFuncs, nluFuncs)
    decoder.io.state := stateMachine.io.state
    
    
    // Weight RF
    val weightRF = new pRF(RFports, weightRFBP, datawidth, addrwidth)
    
    // Mandatory Control
    weightRF.io.write_en    := decoder.io.weightRF_wen
    weightRF.io.read_en     := decoder.io.weightRF_ren
    weightRF.io.waddr       := decoder.io.weightRF_waddr
    weightRF.io.raddr_int   := decoder.io.weightRF_raddr_int
    weightRF.io.raddr_ext   := decoder.io.weightRF_raddr_ext
    
    // Optional Control
    if ( weightRF.io.bp_slct.isDefined ) { weightRF.io.bp_slct.get := decoder.io.weightRF_bp_slct_get }
    
    // Mandatory Outputs
    weightRF.io.wdata := io.weightRF_in
    
    // Optional Outputs
    io.weightRF_2NoC  := weightRF.io.rdata_ext
    
    // Activation RF
    val actvtnRF = new pRF(RFports, actvtnRFBP, datawidth, addrwidth)
    
    // Mandatory Control
    actvtnRF.io.write_en    := decoder.io.actvtnRF_wen
    actvtnRF.io.read_en     := decoder.io.actvtnRF_ren
    actvtnRF.io.waddr       := decoder.io.actvtnRF_waddr
    actvtnRF.io.raddr_int   := decoder.io.actvtnRF_raddr_int
    actvtnRF.io.raddr_ext   := decoder.io.actvtnRF_raddr_ext
    
    // Optional Control
    if ( actvtnRF.io.bp_slct.isDefined ) { actvtnRF.io.bp_slct.get := decoder.io.actvtnRF_bp_slct_get }
    
    // Mandatory Outputs
    actvtnRF.io.wdata := io.weightRF_in
    
    // Optional Outputs
    io.actvtnRF_2NoC     := actvtnRF.io.rdata_ext
       
    val ipuBP = if(aluFuncs.contains("Add") || aluFuncs.contains("Max")) "Firm" else "None" 
    val ipu   = new IPU(RFports, ipuBP, datawidth)
    if (ipu.io.sel.isDefined) { ipu.io.sel.get := decoder.io.ipu_sel_get }
    ipu.io.in1 := weightRF.io.rdata_int
    ipu.io.in2 := actvtnRF.io.rdata_int
    
    val alu = new ALU(aluFuncs, datawidth)
    alu.io.func_slct := decoder.io.alu_func_slct
    alu.io.innr_prod := ipu.io.out
    if(alu.io.weight_bp.isDefined) alu.io.weight_bp.get := ipu.io.bp1.get
    if(alu.io.actvtn_bp.isDefined) alu.io.actvtn_bp.get := ipu.io.bp2.get
    
    val intrnlRF = new pRF(1, intrnlRFBP, datawidth, addrwidth)
    intrnlRF.io.write_en  := decoder.io.intrnlRF_write_en
    intrnlRF.io.read_en   := decoder.io.intrnlRF_read_en
    intrnlRF.io.waddr     := decoder.io.intrnlRF_waddr
    intrnlRF.io.raddr_int := decoder.io.intrnlRF_raddr_int
    intrnlRF.io.raddr_ext := decoder.io.intrnlRF_raddr_ext
    if (intrnlRF.io.bp_slct.isDefined) { intrnlRF.io.bp_slct.get := decoder.io.intrnlRF_bp_slct_get }
    intrnlRF.io.wdata := Mux(decoder.io.intrnlRF_wdata_slct, alu.io.output, io.intrnlRF_in)
    io.intrnlRF_2NoC := intrnlRF.io.rdata_ext
    if(alu.io.rf_feedbk.isDefined) alu.io.rf_feedbk.get := intrnlRF.io.rdata_int
    
    val nlu = new NonlinearUnit(nluFuncs, datawidth)
    nlu.io.fslct := decoder.io.nlu_func_slct
    nlu.io.input     := intrnlRF.io.rdata_int
    io.output        := nlu.io.outpt
    
    // Woot woot
}

defined class nPE

## Future Plans

Basing the nPE off bypasses (i.e. subtractive modifications to a preconfigured topology) is limiting. A constructive approach would be much more general. This should be accomplished through a DSL and much more fully utilizing the functional programming capabilities of Scala.